In [1]:
#-------------------------------------------------------------------------------------JUPYTER NOTEBOOK SETTINGS-------------------------------------------------------------------------------------
from IPython.core.display import display, HTML                                    
display(HTML("<style>.container { width:100% !important; }</style>"))  

/var/folders/ry/454yhlzx6hd15j7rjv4th0lw0000gn/T/ipykernel_95881/2828031236.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import os
import gc
import re
import librosa
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from joblib import dump, load

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Reshape, UpSampling2D, Conv2D, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping 
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import mixed_precision
from tqdm.keras import TqdmCallback
from tensorflow.image import resize

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [3]:
import os
import requests
from pathlib import Path
import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import (
    Dense, InputLayer, Dropout, Conv1D, Flatten, Reshape, MaxPooling1D, BatchNormalization,
    Conv2D, GlobalMaxPooling2D, Lambda, GlobalAveragePooling2D, Input)
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.losses import categorical_crossentropy

In [4]:
x_train, y_train = load('saved_data/data/adversarial-training_medium-masked_data/train_data.joblib')
x_val, y_val = load('saved_data/data/adversarial-training_medium-masked_data/val_data.joblib')
x_test, y_test = load('saved_data/data/adversarial-training_medium-masked_data/test_data.joblib')
print("All data has been loaded properly!")

All data has been loaded properly!


In [5]:
print("Train data:", x_train.shape, x_train.dtype)
print("Validation data:", x_val.shape, x_val.dtype)
print("Test data:", x_test.shape, x_test.dtype)

# Check for any NaN or inf values in your dataset
print("NaNs in train:", np.isnan(x_train).any())
print("NaNs in validation:", np.isnan(x_val).any())
print("NaNs in test:", np.isnan(x_test).any())

print("Infs in train:", np.isinf(x_train).any())
print("Infs in validation:", np.isinf(x_val).any())
print("Infs in test:", np.isinf(x_test).any())

Train data: (63000, 13, 332) float32
Validation data: (13500, 13, 332) float32
Test data: (13500, 13, 332) float32
NaNs in train: False
NaNs in validation: False
NaNs in test: False
Infs in train: False
Infs in validation: False
Infs in test: False


In [6]:
# ONEHOT ENCODING THE LABELS
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# Convert labels to one-hot encoding
y_train_onehot = to_categorical(y_train_encoded)
y_val_onehot = to_categorical(y_val_encoded)
y_test_onehot = to_categorical(y_test_encoded)

In [7]:
# Define the path for weights
WEIGHTS_PATH = 'saved_data/models/edge_impulse_kwt_mobile_weights/kws_mobilenetv2_a35.-mfe-dsp-ver-4_train_c9040_00000_0_batch_size=256,lr=0.0001_2023-01-10_02-23-12.dnu_2048_030.h5'
p = Path(WEIGHTS_PATH)
if not p.exists():
    raise FileNotFoundError(f"Pretrained weights {WEIGHTS_PATH} not found.")

# Define input shape based on x_train data
INPUT_SHAPE = (13, 332, 1)  # Shape of your input data
TARGET_SHAPE = (32, 332, 1)  # Shape required by MobileNetV2

# Reshape x_train to have 4 dimensions
x_train = x_train.reshape((-1, 13, 332, 1))
x_val = x_val.reshape((-1, 13, 332, 1))
x_test = x_test.reshape((-1, 13, 332, 1))

# Resize the input data to match TARGET_SHAPE
x_train_resized = resize(x_train, (32, 332))
x_val_resized = resize(x_val, (32, 332))
x_test_resized = resize(x_test, (32, 332))

# Load MobileNetV2 base model
base_model = MobileNetV2(
    input_shape=TARGET_SHAPE, alpha=0.35,
    weights=None, include_top=False)
base_model.load_weights(WEIGHTS_PATH)
base_model.trainable = False

# Define the number of classes based on the labels
num_classes = len(np.unique(y_train))

# Build the model
model = Sequential()
model.add(InputLayer(input_shape=x_train_resized.shape[1:], name='x_input'))
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.1))
model.add(Dense(2, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model with a higher initial learning rate
initial_learning_rate = 0.001
model.compile(optimizer=Nadam(learning_rate=initial_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Prepare datasets for training
BATCH_SIZE = 512
EPOCHS = 300
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_resized, y_train_onehot)).shuffle(buffer_size=BATCH_SIZE*4).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
validation_dataset = tf.data.Dataset.from_tensor_slices((x_val_resized, y_val_onehot)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-8)
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
callbacks = [reduce_lr, TqdmCallback(), early_stopping_monitor]

# Train the model
model.fit(train_dataset, validation_data=validation_dataset, epochs=EPOCHS, verbose=0, callbacks=callbacks)

# Evaluate the model
test_dataset = tf.data.Dataset.from_tensor_slices((x_test_resized, y_test_onehot)).batch(BATCH_SIZE)
test_loss, test_accuracy = model.evaluate(test_dataset, verbose=2)
print(f'Test accuracy: {test_accuracy:.4f}')

2024-05-22 12:33:58.243358: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-05-22 12:33:58.243386: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-05-22 12:33:58.243390: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-05-22 12:33:58.243410: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-22 12:33:58.243419: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/ciprian/Documents/anaconda/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` inste

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

2024-05-22 12:34:01.949894: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 